In [1]:
import tensorflow as tf
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import numpy as np

from keras.applications.vgg16 import VGG16

img_height=224
img_width=291
batch_size=32

In [2]:
keras.__version__

'2.4.0'

In [3]:
tf.__version__

'2.3.0'